In [3]:
# input file path
d = 'dataset/1_2021-08-05 vf plate/group3-06.czi'

# Importing necessary libraries and defining functions for the analysis:
import numpy as np
from czifile import CziFile
from skimage.filters import threshold_li
import scipy
import matplotlib.pyplot as plt
from skimage.measure import label   
from scipy import ndimage
from fil_finder import FilFinder2D
import astropy.units as u
from skimage.morphology import skeletonize as skeletonize_sci
from os import path
import os
from skimage.transform import resize
from skimage.segmentation import morphological_chan_vese, checkerboard_level_set
from skimage.measure import label, regionprops
from scipy.ndimage import gaussian_filter
from skimage.morphology import local_maxima
import skimage.graph

def getLargestCC(segmentation):
    labels = label(segmentation)
    assert(labels.max() != 0)
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

def modified_max_inscribed_circle(bw, f):    
    D = ndimage.distance_transform_edt(bw)
    Rs = -np.sort(-D, axis=None)
    R = Rs[0]
    RInds = np.argsort(-D, axis=None)
    RInds = RInds[Rs >= f*R]
    [cy, cx] = np.unravel_index(RInds, D.shape)
    return R, cx, cy

def create_circular_mask(h, w, center=None, radius=None):
    if center is None: 
        center = (int(w/2), int(h/2))
    if radius is None: 
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

with CziFile(d) as im:
    image_array = im.asarray()

# Defining the correct channels into new arrays based on the array shape
im_sprout = image_array[0,0,0,0,:,:,:,0]
im_nuclei = image_array[0,0,0,1,:,:,:,0]

# median filtering and taking the maximum projection of the sprout channel
med_sprout = im_sprout
for _ in range(2):
    med_sprout = scipy.ndimage.median_filter(med_sprout, size=(3,3,3))
max_projection = np.max(med_sprout, axis=0)

# Preprocessing steps and skeletonization
from skimage.filters import threshold_otsu
bw_mproj = max_projection >= threshold_otsu(max_projection)
largest_cc = getLargestCC(bw_mproj)
bw = ndimage.binary_fill_holes(largest_cc) * 1
med_bw = scipy.ndimage.median_filter(bw, size=(6,6))
skeleton_sci = skeletonize_sci(med_bw)

R, cx, cy = modified_max_inscribed_circle(bw, f=0.9)
sz = bw.shape
bead = np.zeros(sz, dtype=bool)
if len(cx) > 200: sp = 10
else: sp = 20 
for i in range(0, len(cx), sp):
    # You can change radius to R*n in order to get the correct seperation of sprouts
    circ_ = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*1.1)
    bead = np.logical_or(bead, circ_)

bw_rmv_bead = np.logical_and(med_bw, np.logical_not(bead))
labels = label(bw_rmv_bead)
lbls = np.unique(labels[(skeleton_sci).astype(np.bool_)])
lbls_inx = np.nonzero(lbls)[0]
L = 1
sprouts = np.zeros(sz, dtype=np.uint8)
for i in lbls_inx:
    sprouts[labels==lbls[i]] = L
    L += 1
med_n_sp = (med_sprout[:, ::2, ::2]).astype(np.float64)
    
# Creating a mask which does not include the bead part only includes the sprout part for segmentation
sprouts_2D = (resize(sprouts, output_shape=(med_n_sp.shape[1],med_n_sp.shape[2]), order=0)).astype(np.bool_)
med_n_sp_copy = med_n_sp.copy()
for i in range(med_n_sp_copy.shape[0]):
    med_n_sp_copy[i,:,:] = sprouts_2D

# Chan-Vese segmentation part. You can increase iterations if there are extra parts segmented
# or decrease iterations if there are some target parts not segmented
bw_sp = morphological_chan_vese(image=med_n_sp, num_iter=10, init_level_set=med_n_sp_copy, lambda1=1, lambda2=1)
np.count_nonzero(bw_sp)

import napari
viewer = napari.view_image(med_n_sp, name='med_n_sp')
napari.run()  

viewer.add_labels(bw_sp)

<Labels layer 'bw_sp' at 0x21411141120>

Exception in callback BaseAsyncIOLoop._handle_events(7376, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(7376, 1)>
Traceback (most recent call last):
  File "C:\Users\musta\.conda\envs\mb22\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\musta\.conda\envs\mb22\lib\site-packages\tornado\platform\asyncio.py", line 206, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\musta\.conda\envs\mb22\lib\site-packages\zmq\eventloop\zmqstream.py", line 577, in _handle_events
    self._handle_recv()
  File "C:\Users\musta\.conda\envs\mb22\lib\site-packages\zmq\eventloop\zmqstream.py", line 606, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\musta\.conda\envs\mb22\lib\site-packages\zmq\eventloop\zmqstream.py", line 556, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\musta\.conda\envs\mb22\lib\site-packages\jupyter_client\threaded.py", line 123, in _handle_recv
    msg_list = se